In [ ]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.expr.types import (
    ArrayValue,
    FloatingValue,
    IntegerValue,
    StringValue,
    StructValue,
)

ibis.options.interactive = True


: 

We are going to dedupe the PATSTAT dataset. Every record represents a patent filed,
and our task is to determine which one came from the same inventor.
The end goal is to add a column called `inventor_id` to each patent.
This dataset contains a ground truth label, so we can evaluate how well we did.

`Mismo` contains this as an included dataset so it is easy to get started.
The returned dataset is an Ibis table, which is a lazy representation of a SQL table.
It is similar to a pandas dataframe, but has a few properties that make it much
better for the record linkage use case:

- Since it is SQL backed, it can handle datasets that are larger than memory, in
  the many millions of rows.
- Computation is performed by the powerful SQL backend of your choice: Google BigQuery,
  Apache Spark, Snowflake, etc. For this demo, we use DuckDB, which is a 
  state-of-the-art SQL engine based around a columnar data model
  (ie oriented towards the bulk operations of record linkage)
- Ibis is strongly typed, has a full API, is well-documented, and has good
  integration with the rest of the python data science ecosystem.


In [ ]:
from mismo.datasets import load_patents  # noqa: E402

patents = load_patents()
print(patents.count())
patents


: 

In [ ]:
from mismo.plot import plot_distributions  # noqa: E402

plot_distributions(patents)


: 

Let's clean this up a bit:
- clean up whitespace
- convert the `coauthors` and `classes` columns to actual arrays (they really represent sets)

Each element in `classes` is a 4-character IPC technical code that is like a tag
for the patent. Similar patents will have similar tags.


In [ ]:
from mismo.clean.strings import norm_whitespace  # noqa: E402


def clean_names(names: StringValue) -> StringValue:
    names = norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Now have to do whitespace fixup again
    names = norm_whitespace(names)
    return names


def parse_list(s: StringValue) -> ArrayValue:
    return s.upper().split("**").map(norm_whitespace).sort()


cleaned = patents.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=clean_names(_.name),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=parse_list(_.coauthors.nullif("NONE")),
    classes=parse_list(_.classes),
)


: 

Then, let's add some features. The binned coordinates will be used in the blocking step,
so that locations in the same lat/lng bin will be compared to each other.

We also generate some features based on the `name` column.

In [ ]:
def bin_lat_lon(lat: FloatingValue, lon: FloatingValue) -> StructValue:
    """Bin a latitude or longitude to 0.1 degree precision, which is ~6 miles.

    If both are null, return null.

    (52.35, 4.916667 -> (524, 49)
    """

    def _bin_coord(coord: FloatingValue) -> IntegerValue:
        return (coord.round(1) * 10).cast("int16").fillna(0)

    result = ibis.struct(
        {
            "lat_hash": _bin_coord(lat),
            "lon_hash": _bin_coord(lon),
        }
    )
    both_null = lat.isnull() & lon.isnull()
    return both_null.ifelse(ibis.null(), result)


featured = cleaned.mutate(
    name_tokens=_.name_cleaned.split(" ").map(norm_whitespace).sort(),
    name_first3=_.name_cleaned[0:3],
    coords_hashed=bin_lat_lon(_.latitude, _.longitude),
)
featured


: 

OK, now it's time to block! This is where we generate comparisons between records.
If we were naive and generated all possible comparisons from N record,
you would end up with N^2 comparisons. For our small dataset of ~2000 records
we would be able to get away with this, but for datasets much larger than this
it would be infeasible.

In [ ]:
from mismo.block import BlockingRule, BlockingRules  # noqa: E402

rules = BlockingRules(
    BlockingRule(
        "Coordinates Close",
        lambda left, right: left.coords_hashed == right.coords_hashed,
    ),
    BlockingRule(
        "Name First 3", lambda left, right: left.name_first3 == right.name_first3
    ),
    BlockingRule(
        "Coauthors Exact", lambda left, right: left.coauthors == right.coauthors
    ),
    BlockingRule("Classes Exact", lambda left, right: left.classes == right.classes),
)

featured = featured.cache()
blocked = rules.block(featured, featured, labels=True)
blocked = blocked.cache()
blocked


: 

: 

The result of that was the two tables joined together, with a `_l` added
to all the columns from the left table, and a `_r` added to all the columns
from the right table. In addition, there is a column `blocking_rules` that
tells us which blocking rules were used to generate the pair.

By blocking, we reduced the number of needed pairs by a large factor.
In larger datasets, and with better blocking rules, this would be even more!


In [ ]:
from mismo import metrics  # noqa: E402

n_comparisons = blocked.count().execute()
n_naive = metrics.n_naive_comparisons(featured)
reduction_ratio = n_comparisons / n_naive
n_naive, n_comparisons, reduction_ratio


: 

OK, now that we have our candidate pairs generated, let's actually do the
comparing of pairs. There are many ways to do this, but one of the most common
is to generate a set of Comparison objects, each of which represents a
measurement of similarity based on some dimension. Each Comparison is
composed of Levels, each of which is a level of agreement:

- How well the names match
    - exact match
    - the first three letters match
    - everything else (this is implicit, we don't actually pass this feature)
- How well the set of classes match
    - exact match
    - the [jaccard index](https://en.wikipedia.org/wiki/Jaccard_index) of the sets of classes is above 50%
    - everything else
- How well the set of coauthors match
    - exact match
    - the jaccard index of the sets of coauthors is above 50%
    - everything else
- How well the locations match
    - exact match
    - within 100 km
    - one or both coords are NULL
    - everything else 

In [ ]:
from mismo.compare import (  # noqa: E402
    Comparison,
    ComparisonLevel,
    Comparisons,
    distance_km,
    exact_level,
    jaccard,
)

name_comparison = Comparison(
    name="Name",
    levels=[
        exact_level("name_cleaned"),
        exact_level("name_first3"),
    ],
)

classes_comparison = Comparison(
    name="Classes",
    levels=[
        exact_level("classes"),
        ComparisonLevel(
            name="Classes 50% Jaccard",
            condition=lambda table: jaccard(table.classes_l, table.classes_r) >= 0.5,
        ),
    ],
)

coauthors_comparison = Comparison(
    name="Coauthors",
    levels=[
        exact_level("coauthors"),
        ComparisonLevel(
            name="Coauthors 50% Jaccard",
            condition=lambda table: jaccard(table.coauthors_l, table.coauthors_r)
            >= 0.5,
        ),
    ],
)

coords_comparison = Comparison(
    name="Coords",
    levels=[
        ComparisonLevel(
            name="Coords match",
            condition=lambda table: (table.latitude_l == table.latitude_r)
            & (table.longitude_l == table.longitude_r),
        ),
        ComparisonLevel(
            name="Coords within 100km",
            condition=lambda table: distance_km(
                lat1=table.latitude_l,
                lon1=table.longitude_l,
                lat2=table.latitude_r,
                lon2=table.longitude_r,
            )
            <= 100,
        ),
        ComparisonLevel(
            name="One or both coords missing",
            condition=lambda table: table.coords_hashed_l.isnull()
            | table.coords_hashed_r.isnull(),
        ),
    ],
)

comparisons = Comparisons(
    name_comparison,
    classes_comparison,
    coauthors_comparison,
    coords_comparison,
)
compared = comparisons.label_pairs(blocked, how="name")
compared = compared.cache()
compared


: 

The result above is the blocked table, with a column added for every `Comparison`.
The value of each column is the level that the record pair matched at.
For example, there is now a "Name" column,
filled with values like "exact_name_cleaned", "exact_name_first3", etc.

We can plot these features to see how common each level of match is,
and which levels are related to each other. You can zoom in by dragging
on the bottom minimap plot.

In [ ]:
from mismo.compare import plot  # noqa: E402

plot(compared, comparisons)


: 

We can use this to improve our Comparisons. Notice how it is fairly common for
Classes, Coords, and Name to match exactly, but Coauthors to have the `else` level?
These comparisons are probably true matches, if we inspect them we can see the
coauthors of `['REYNOLDS DUDLEY JOHN']` paired with `['REYNOLDS JOHN DUDLEY']`
getting classified as `else` because the names don't match exactly. We should probably
adjust our Coauthors rule to account for this, but to keep this short we will not here.

In [ ]:
compared.filter(
    [
        _.Classes == "exact_classes",
        _.Coauthors == "else",
        _.Coords == "Coords match",
        _.Name == "exact_name_cleaned",
    ]
)


: 

Now that we have our features, we can use the Fellegi-Sunter model to train weights
for each of these features. This is a probabilistic model that is based on the concept
of odds. When you see an exact match on name, that increases the odds of a match
by some amount, maybe 50x. When you see a non-match on name, that decreases the odds
of a match by some amount, maybe 0.1x. We can either train this from labeled data,
or we can use unlabeled data using an algorithm called "Expectation Maximization".

In [ ]:
from mismo.compare.fs import train_comparisons  # noqa: E402

weights = train_comparisons(comparisons, featured, featured, max_pairs=10_000, seed=42)
weights.plot()


: 

Use the weights to score the record pairs, findng the odds for each
Comparison, and then combining them into an overall odds for the record pair.

In [ ]:
scored = weights.score(compared)
scored = scored.cache()
scored


: 

Look at the different combinations of comparisons that we found, and see which ones
led to the best match, and which ones to the worst.

Unsurprisingly, the exact match levels have the highest odds, and the
ELSE levels have the lowest. The other levels are somewhere in between.

In [ ]:
comparison_columns = [comp.name for comp in comparisons]
vector_counts = scored.group_by(comparison_columns).agg(
    n_pairs=_.count(),
    odds=_.odds.arbitrary(),
)
best = vector_counts.order_by(_.odds.desc()).head(5)
worst = vector_counts.order_by(_.odds.asc()).head(5)
best, worst


: 

Let's plot all of these to try to find a threshold for what to count as a match

In [ ]:
import altair as alt  # noqa: E402

alt.Chart(vector_counts.execute()).mark_bar().encode(
    x="odds",
    y=alt.Y(
        "n_pairs",
        scale=alt.Scale(type="log"),
    ),
    color="odds",
    tooltip=[
        "odds",
        "Name",
        "Classes",
        "Coauthors",
        "Coords",
    ],
)


: 

It looks like an odds of 1500 seems to separate the comparisons into two nice clusters.
If I hover over the above chart, I can see that pretty much all the ELSE comparisons
are in the low cluster, and all the SAME comparisons are in the high cluster.

In [ ]:
odds_threshold = 1500
(scored.odds >= odds_threshold).value_counts()


: 

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [ ]:
from mismo.cluster import connected_components  # noqa: E402

links = scored[_.odds >= odds_threshold]
links = links.cache()
print(links.count())
labels = connected_components(links, nodes=featured.record_id)
print(labels.count())
labels


: 

Now let's evaluate how good our labeling is. Mismo wraps all of the evaluation
metrics from sklearn, so we can use them with Ibis Tables.

In [ ]:
labels_true = patents.select("record_id", label=_.label_true)
labels_pred = labels.select("record_id", label=_.component)
metrics.adjusted_rand_score(labels_true, labels_pred)


: 

In [ ]:
metrics.homogeneity_score(labels_true, labels_pred)


: 

The high homogeneity means we have a high precision, and don't have a lot of false-links

In [ ]:
metrics.completeness_score(labels_true, labels_pred)


: 

The low completeness score means we have low recall, and are missing a lot of true-links